In [1]:
from tensorflow.python import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas

/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
# Каталог с данными для тестирования
testDir = './bigpic/transformed/test'
# Размеры изображения
img_width, img_height = 100, 100
input_shape = (img_width, img_height, 3)
# Количество изображений для обучения
nb_train_samples = 656
# Количество изображений для проверки
nb_validation_samples = 83
# Количество изображений для тестирования
nb_test_samples = 83

In [3]:
classes = ['круассан','полки','пончик', 'завиток',]

In [4]:
def SaltAndPaper (im, probability):
    # probability of the noise 
    out = np.random.randint(0,probability, im.shape)
    out = out + im
    out[out>255]=255
    #out = out.astype(np.uint8)
    return out

def ColorBalance(im, probability):
    colorB = np.random.random() * (1-probability) + probability
    colorG = np.random.random() * (1-probability) + probability
    colorR = np.random.random() * (1-probability) + probability
    out = im.copy()
    out[:,:,0] = im[:,:, 0] * colorB
    out[:,:,1] = im[:,:, 1] * colorG
    out[:,:,2] = im[:,:, 2] * colorR
    return out

def BoxFilter(im, ksize):
    BoxFilter_image = cv2.boxFilter(im, 0, (ksize,ksize))
    #BoxFilter_image
    return BoxFilter_image

def Transform(img):
    #img = ColorBalance(img, 0.5)
    #img = BoxFilter(img, 3)
    img = SaltAndPaper(img, 20)
    img = img.astype(np.float32)
    return img

In [5]:
# https://keras.io/preprocessing/image/
datagen = ImageDataGenerator(rescale=1./255., 
                             rotation_range=45,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=0.2,
                             shear_range=0.2,
                             brightness_range=[0.5, 1.0],
                             channel_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             fill_mode="nearest",
                             preprocessing_function=Transform
                            )

In [6]:
test_df = pandas.read_csv("./bigpic/transformed/test.csv")

# https://keras.io/preprocessing/image/#flow_from_dataframe

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=None,
        x_col="filename",
        y_col="label",
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='categorical')


Found 83 validated image filenames belonging to 4 classes.


In [17]:
# загрузка предварительно обученной модели
#model = keras.models.load_model('modelWork.hdf5')

model = keras.models.load_model('./saveModelWeightStep/24-0.9457-0.9821 workModel.hdf5')

In [18]:
model.summary()
#model.layers

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0

In [19]:
scores = model.evaluate_generator(test_generator, nb_test_samples)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Точность на тестовых данных: 96.39%


In [10]:
model.summary()
#model.layers

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0

In [15]:
# из csv-файла с сохраненными моделями тестируем каждую
spisokModels = pandas.read_csv("./SavedModel/modelslist.csv", header = None)
for m in range(len(spisokModels)):
    modelName = spisokModels.iloc[m,0].split('/')
    print(modelName[-1])
    model = keras.models.load_model(spisokModels.iloc[m,0])
    scores = model.evaluate_generator(test_generator, nb_test_samples)
    print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

01-0.9443-0.8929.hdf5
Точность на тестовых данных: 95.18%
02-0.9697-0.9643.hdf5
Точность на тестовых данных: 96.39%
03-0.9697-0.8750.hdf5
Точность на тестовых данных: 95.18%
04-0.9777-0.9107.hdf5
Точность на тестовых данных: 95.18%
05-0.9503-0.9286.hdf5
Точность на тестовых данных: 96.39%


In [20]:
# загружаем созданный csv-файл со списком тестовых картинок и отображаем результаты теста по каждой
spisoktest = pandas.read_csv("./bigpic/transformed/test.csv")
for k in range(len(spisoktest)):
    picName = spisoktest.iloc[k,0].split('/')
    print(k+1, picName[-1])
    im = spisoktest.iloc[k,0]
    img = image.load_img(im, target_size=(img_width, img_height))
    # Преобразуем картинку в массив
    x = image.img_to_array(img)
    x = x.reshape(1, x.shape[0],x.shape[1],x.shape[2])
    # Нормализуем изображение
    x /= 255
    prediction = model.predict(x)
    prediction
    prediction = np.argmax(prediction)
    print("Номер класса:", prediction)
    print("Название класса:", classes[prediction]) 

1 cruassan.234.jpg
Номер класса: 3
Название класса: завиток
2 cruassan.235.jpg
Номер класса: 0
Название класса: круассан
3 cruassan.236.jpg
Номер класса: 0
Название класса: круассан
4 cruassan.237.jpg
Номер класса: 0
Название класса: круассан
5 cruassan.238.jpg
Номер класса: 3
Название класса: завиток
6 cruassan.239.jpg
Номер класса: 0
Название класса: круассан
7 cruassan.240.jpg
Номер класса: 0
Название класса: круассан
8 cruassan.241.jpg
Номер класса: 0
Название класса: круассан
9 cruassan.242.jpg
Номер класса: 0
Название класса: круассан
10 cruassan.243.jpg
Номер класса: 0
Название класса: круассан
11 cruassan.244.jpg
Номер класса: 0
Название класса: круассан
12 cruassan.245.jpg
Номер класса: 0
Название класса: круассан
13 cruassan.246.jpg
Номер класса: 0
Название класса: круассан
14 cruassan.247.jpg
Номер класса: 0
Название класса: круассан
15 cruassan.248.jpg
Номер класса: 0
Название класса: круассан
16 cruassan.249.jpg
Номер класса: 0
Название класса: круассан
17 cruassan.250.jpg